In [6]:
%pip install langchain langchain-core langgraph

Note: you may need to restart the kernel to use updated packages.


In [59]:
#from define_state_and_llm import State, llm, llm_with_tools
from langchain_core.prompts import PromptTemplate
import json
from langgraph.prebuilt import ToolNode
from typing import TypedDict, List
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
import requests
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from tavily import TavilyClient

In [10]:
load_dotenv()

class State(TypedDict):
    interest_rate: float | str
    treasury_yield: float | str
    market_rate: float
    num_tool_calls: int
    path: List[str]
    recommendation: str
    #credit_score: int
    #closing_costs: int
    #current_payment: int

llm = ChatOpenAI(model="gpt-4o-mini",
                 api_key=os.getenv("OPENAI_API_KEY"))

In [11]:
def get_treasury_10yr_yield() -> float:
    """"
    Gets the 10 year treasury yield value.
    """

    url = "https://quote.cnbc.com/quote-html-webservice/quote.htm"

    headers = {
    "User-Agent": (
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/118.0.5993.89 Safari/537.36"
    )
    }

    params = {
        "noform": "1",
        "partnerId": "2",
        "fund": "1",
        "exthrs": "0",
        "output": "json",
        "symbols": "US10Y"
    }

    resp = requests.get(url, params=params, headers=headers, timeout=8)
    resp.raise_for_status()
    data = resp.json()

    try:
        quotes = data["QuickQuoteResult"]["QuickQuote"]
        value = quotes[0]["last"]
        str_value = str(value).strip().rstrip("%")
        return float(str_value)
    except Exception as e:
        raise ValueError(f"Unexpected CNBC payload shape or symbol missing: {e}") from e

In [14]:
@tool
def get_treasury_10yr_yield_for_agent() -> float | str:
    """Gets the 10 year treasury yield value."""
    return get_treasury_10yr_yield()

In [31]:
llm_with_tools = llm.bind_tools([get_treasury_10yr_yield_for_agent])

In [43]:
def treasury_yield_agent(state: State) -> dict:
    """Agent gets the latest news articles and summarizes how mortgage rates are at the moment.
    It will also review the 10 year treasury yield rate and see if it's good."""
    prompt = PromptTemplate(template="""You are an expert on treasury yields. You should review the 
                            current 10 Year Treasury Yield rate by calling the `get_treasury_10yr_yield_for_agent` 
                            tool to get the current 10-year Treasury yield value.
                            """)
    resp = llm_with_tools.invoke(prompt.format())
    print(f"\n\n\n*******RESP: {resp}*******\n\n\n")

    tool_node = ToolNode([get_treasury_10yr_yield_for_agent])
    tool_result = tool_node.invoke({"messages": [resp]})
    value = float(tool_result["messages"][0].content)

    print(f"\n\n\n*******tool_result: {tool_result}*******\n\n\n")

    print(f"\n\n\n*******tool_result: {value}*******\n\n\n")

    return state

In [ ]:
    #  resp = llm_with_tools.invoke(prompt.format())
    
    # # Check if LLM wants to call tools
    # if resp.tool_calls:
    #     tool_result = tool_node.invoke({"messages": [resp]})
    #     # Send tool results back to LLM for final response
    #     final_resp = llm_with_tools.invoke([resp] + tool_result["messages"])
    #     content = final_resp.content
    # else:
    #     content = resp.content
    
    # # Parse the JSON response
    # import json
    # parsed = json.loads(content)
    
    # state["treasury_yield"] = parsed["treasury_yield"

In [44]:
workflow = StateGraph(State)
workflow.add_node("treasury_yield_agent_node", treasury_yield_agent)

workflow.set_entry_point("treasury_yield_agent_node")
workflow.add_edge('treasury_yield_agent_node', END)

app=workflow.compile()

In [45]:
initial_state = {
    "interest_rate": 9.0,
    "treasury_yield": 0.0,
    "market_rate": 0.0,
    "recommendation": "",
    "num_tool_calls": 0,
    "path": [""]
}

result = app.invoke(initial_state)




*******RESP: content='' additional_kwargs={'tool_calls': [{'id': 'call_kECpLSIzxJmijdCIR5iXkAA4', 'function': {'arguments': '{}', 'name': 'get_treasury_10yr_yield_for_agent'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 101, 'total_tokens': 120, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--739925ba-0462-4b09-a8ed-458c3d9600bc-0' tool_calls=[{'name': 'get_treasury_10yr_yield_for_agent', 'args': {}, 'id': 'call_kECpLSIzxJmijdCIR5iXkAA4', 'type': 'tool_call'}] usage_metadata={'input_tokens': 101, 'output_tokens': 19, 'total_tokens': 120, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio'

In [79]:
def get_rates_search_tool() -> str:
    """Get the average mortgage interest rate."""

    tavily_client = TavilyClient(api_key=os.getenv('TAVILY_API_KEY'))

    response = tavily_client.search(
        query="""What is the current average mortgage interest rate people in the United States are receiving?" \
        "Provide the answer as a number with 2 decimal places. E.g., 6.55.
        ONLY provide the number without any additional text. ONLY return a single numerical value.""",
        topic="finance",
        search_depth="basic",
        max_results=3,
        time_range="week",
        include_answer=True #Include an LLM-generated answer to the provided query. 
    )

    answer = response["answer"]
    return answer

In [80]:
@tool
def get_rates_search_tool_for_agent() -> str | float:
    """Get the average mortgage interest rate."""
    return get_rates_search_tool()

In [81]:
llm_with_tools = llm.bind_tools([get_treasury_10yr_yield_for_agent, get_rates_search_tool_for_agent])

In [125]:
def market_expert_agent(state: State) -> dict:
    prompt = PromptTemplate(template="""
                            You are a mortgage market expert. You should summarize some recent articles to get 
                            an average mortgage interest rate people are seeing right now by 
                            calling the `get_rates_search_tool_for_agent`.
                            """)
    prompt_str = prompt.format()
    resp = llm_with_tools.invoke(prompt_str)

    # Check if LLM wants to call tools
    if resp.tool_calls:
        # Execute the call to get_rates_search_tool_for_agent
        tool_result = tool_nodes.invoke({"messages": [resp]})
        message = tool_result["messages"][0].content

        # Extract the single value from the text
        follow_on_prompt = f"""Extract the average mortgage interest rate value from this body of text: {message}
                              You must ONLY return the numerical value up to two decimal places.
                              Example answer: 5.32"""
        updated_resp = llm.invoke(follow_on_prompt)
        value = float(updated_resp.content)
        print("===SUCCESSFULLY EXECUTED MARKET RESEARCH AGENT TOOL CALL===")
    else:
        value = 0.0

    print(f"\nMARKET RATE: {value}\n")
    state["market_rate"] = value
    state["num_tool_calls"] += 1
    state["path"].append("market_expert_agent")
    return state


In [92]:
workflow = StateGraph(State)
workflow.add_node("market_expert_node", market_expert_agent)

workflow.set_entry_point("market_expert_node")
workflow.add_edge('market_expert_node', END)

app=workflow.compile()

In [93]:
initial_state = {
    "interest_rate": 9.0,
    "treasury_yield": 0.0,
    "market_rate": 0.0,
    "recommendation": "",
    "num_tool_calls": 0,
    "path": [""]
}

result = app.invoke(initial_state)

As of the current date, the average mortgage interest rate people in the United States are receiving is 6.55 percent. This figure is derived from the analysis provided by Investopedia, which reports the average 30-year fixed rate at 6.50 percent, compared with 7.15 percent in mid-May, based on their daily Zillow rate data.
UPDATED RESPONSE: 6.55


In [94]:
data_output_dict = {
        "recommendation": "The agentic refinance tool recommendation is:",
        "num_tool_calls": "The total number of tools called was:",
        "path": "The path the agentic workflow took was:",
    }

In [98]:
data_output_dict.keys()

dict_keys(['recommendation', 'num_tool_calls', 'path'])

In [101]:
market_interst_rate = 6.125
mortgage_balance = 700000

In [103]:
remaining_term_years = 30
    # Convert to monthly rate
r = (market_interst_rate / 100) / 12
n = int(remaining_term_years * 12)  # total remaining monthly payments
    
    # Compute new monthly payment using amortization formula
if r == 0:
        # Zero-interes  at edge case
    mortgage_balance / n
    
new_payment = mortgage_balance * (r * (1 + r)**n) / ((1 + r)**n - 1)

In [104]:
new_payment

4253.273777064836

In [156]:
@tool
def calculate_estimates_and_breakeven_for_agent(#interest_rate: float,
                                      current_payment: float, 
                                      mortgage_balance: float, 
                                      market_rate: float) -> tuple[float, float]:
    """Calculate the new monthly mortgage payment and the break-even period on the closing costs"""
    ### New Payment Calculation ###
    remaining_term_years = 30
    # Convert to monthly rate
    r = (market_rate / 100) / 12
    n = int(remaining_term_years * 12)  # total remaining monthly payments
    # Compute new monthly payment using amortization formula
    if r == 0:
        # Zero-interest edge case
        return mortgage_balance / n
    # New Monthly Mortgage Payment (P&I only)
    new_payment = mortgage_balance * (r * (1 + r)**n) / ((1 + r)**n - 1)

    ### Break Even Calculation ###
    estimated_closing_costs = mortgage_balance * 0.01
    monthly_savings = current_payment - new_payment
    break_even = estimated_closing_costs / monthly_savings
    
    return new_payment, monthly_savings, break_even

In [ ]:
# @tool
# def calculate_estimates_and_breakeven_for_agent(current_payment: float, 
#                                       mortgage_balance: float, 
#                                       market_rate: float) -> tuple[float, float]:
#     """Calculate the user's estimated new payment and their breakeven point."""
#     return calculate_estimates_and_breakeven(current_payment: float, mortgage_balance: float, 
#                                       market_rate: float)

In [160]:
tool_nodes = ToolNode([get_treasury_10yr_yield_for_agent, 
                       get_rates_search_tool_for_agent,
                       calculate_estimates_and_breakeven_for_agent])

In [161]:
llm_with_tools = llm.bind_tools([get_treasury_10yr_yield_for_agent, 
                                 get_rates_search_tool_for_agent,
                                 calculate_estimates_and_breakeven_for_agent])

In [ ]:
# Agent #3
def calculator_agent(state: State) -> dict:
    """Agent intakes the user's current principal and interest payment as well as a calculated estimated principal 
    and interest payment based their loan amount. The agent will get a difference in payment (savings) between the current
    and estimated payment. The estimated closing costs will also be calculated. The estimated closing costs divided by savings is the break even
    calculation that will be displayed."""

    prompt = PromptTemplate(
    input_variables=["current_payment", "mortgage_balance", "market_rate"],
    template="""
    You are an expert on calculating new mortgage payments and break-even points.

    You MUST call the tool `calculate_estimates_and_breakeven_for_agent` 
    and pass it exactly these three arguments:

    - current_payment: {current_payment}
    - mortgage_balance: {mortgage_balance}
    - market_rate: {market_rate}

    Your ONLY job is to call the tool with:

    {{
        "current_payment": {current_payment},
        "mortgage_balance": {mortgage_balance},
        "market_rate": {market_rate}
    }}

    After the tool returns, output ONLY valid JSON of the form:
    {{
        "new_payment": <float>,
        "monthly_savings": <float>,
        "break_even": <float>
    }}
    """
    )
    final_prompt = prompt.invoke({
        "current_payment": state['current_payment'],
        "mortgage_balance": state['mortgage_balance'],
        "market_rate": state['market_rate']
        })
    
    resp = llm_with_tools.invoke(final_prompt)

    # Check if LLM wants to call tools
    if resp.tool_calls:
        # Execute the call to tool
        tool_result = tool_nodes.invoke({"messages": [resp]})
        values = json.loads(tool_result["messages"][0].content)

        state['new_payment'] = values[0]
        state['monthly_savings'] = values[1]
        state['break_even'] = values[2]
        print("===SUCCESSFULLY EXECUTED CALCULATOR AGENT TOOL CALL===")
    else:
        values=[0,0,0]
        
    #return State
    print(f"CALCULATOR AGENT RESPONSE: {values}")

In [180]:
load_dotenv()

class State(TypedDict):
    interest_rate: float | str
    treasury_yield: float | str
    market_rate: float
    num_tool_calls: int
    path: List[str]
    current_payment: float
    mortgage_balance: float
    new_payment: float
    monthly_savings: float
    break_even: float
    recommendation: str
    #credit_score: int
    #closing_costs: int
    #current_payment: int

llm = ChatOpenAI(model="gpt-4o-mini",
                 api_key=os.getenv("OPENAI_API_KEY"),
                 temperature=0.0)

In [181]:
def condition(state: State) -> str:
    if state["market_rate"] > state["interest_rate"]:
        return "END"
    else:
        return "CONTINUE"

workflow = StateGraph(State)
workflow.add_node("market", market_expert_agent)
workflow.add_node("calculator", calculator_agent)

workflow.set_entry_point("market")
workflow.add_edge("market", "calculator")
workflow.add_edge("calculator", END)

app=workflow.compile()

In [182]:
initial_state = {
    "interest_rate": 9.0,
    "treasury_yield": 0.0,
    "market_rate": 0.0,
    "num_tool_calls": 0,
    "path": [""],
    "recommendation": "",
    "current_payment": 5000,
    "mortgage_balance": 650000,
    "new_payment": 0,
    "monthly_savings": 0,
    "break_even": 0
}

result = app.invoke(initial_state)

===SUCCESSFULLY EXECUTED MARKET RESEARCH AGENT TOOL CALL===

MARKET RATE: 6.31

===SUCCESSFULLY EXECUTED CALCULATOR AGENT TOOL CALL===

 4027.5611204167635

 972.4388795832365

 6.684224722468667
CALCULATOR AGENT RESPONSE: [4027.5611204167635, 972.4388795832365, 6.684224722468667]
